# RAG with Excel Spreadsheet using LlamaPrase

<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/parse/excel/dcf_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook constructs a RAG pipeline over a simple DCF template [here](https://eqvista.com/app/uploads/2020/09/Eqvista_DCF-Excel-Template.xlsx).



## Setup

We first setup and load the data. If you haven't already, [download the template](https://eqvista.com/app/uploads/2020/09/Eqvista_DCF-Excel-Template.xlsx) and name it `dcf_template.xlxs` locally.

In [ ]:
%pip install llama-index
%pip install llama-cloud-services

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_cloud_services import LlamaParse

# api_key = "llx-"  # get from cloud.llamaindex.ai

In [ ]:
parser = LlamaParse(
    # api_key=api_key,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",
)
docs = parser.load_data("./dcf_template.xlsx")
# docs_txt = LlamaParse(result_type="text").load_data("./dcf_template.xlsx")

Started parsing the file under job_id cac11eca-d5da-4d46-90e6-321f40e11611
Started parsing the file under job_id cac11eca-5450-4847-9da0-fa6879c4cf3a


In [ ]:
print(docs[0].get_content())

# Cover Page

|Thank you for downloading our DCF Model excel template. This DCF Model excel template helps you to value your business using Discounted Free Cash Flow or DCF Method. |                                                                                                                                                                                  |
|----------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|                                                                                                                                                                      |                                                                                                                          

## Configure LLM, Setup Basic Summary Engine

We setup a basic summary engine which retrieves the entire document as context to put into the prompt.

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-4-turbo-preview")
Settings.llm = llm

In [ ]:
from llama_index.core import SummaryIndex

index = SummaryIndex.from_documents(docs)
# index = SummaryIndex.from_documents(docs_txt)

query_engine = index.as_query_engine()

## Define Baseline

Let's define a baseline query engine over this data, using a naive parser (our PandasExcelReader, available on LlamaHub).

In [ ]:
!pip install llama-index-readers-file
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 5.0 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
from llama_index.readers.file import PandasExcelReader
import importlib
from pathlib import Path

base_reader = PandasExcelReader()
base_docs = base_reader.load_data(Path("dcf_template.xlsx"))

In [ ]:
print(base_docs[1].get_content())

Discounted Cash Flow Excel Template           
           
Here is a simple discounted cash flow excel template for estimating your company value based on this income valuation approach           
           
Instructions:           
1) Fill out the two assumptions in yellow highlight           
2) Fill in either the 5 year or 3 year weighted average figures in yellow highlight           
           
           
           
           
Assumptions           
Tax Rate 0.2          
Discount Rate 0.15          
           
5 Year Weighted Moving Average           
Indication of Company Value 242995.4347636059          
           
3 Year Weighted Moving Average           
Indication of Company Value 158651.0723286644          
           
 5 Year Weighted Moving Average          
 Past Years     Forecasted Future Years     
 Year 1 Year 2 Year 3 Year 4 Year 5 Year 6 Year 7 Year 8 Year 9 Year 10 Terminal Value
Pre-tax income 50000 55000 45000 52000 60000      
Income Taxes 10000 11000 900

In [ ]:
from llama_index.core import SummaryIndex

base_index = SummaryIndex.from_documents([base_docs[1]])

base_query_engine = base_index.as_query_engine()

## Ask Questions over this Data

Let's now ask questions over this data, using both the LlamaParse-powered pipeline and naive pipeline.

In [ ]:
query_str = "Tell me about the income taxes in the past years (year 3-5) for the 5 year WMA table"
response = query_engine.query(query_str)
base_response = base_query_engine.query(query_str)

In [ ]:
print("******* LlamaParse RAG *******")
print(str(response))
print("******* Naive RAG *******")
print(str(base_response))

******* LlamaParse RAG *******
The income taxes in the past years (year 3 to 5) for the 5-year Weighted Moving Average table were $9,000.00 in Year 3, $10,400.00 in Year 4, and $12,000.00 in Year 5.
******* Naive RAG *******
The income taxes in the past years (year 3-5) for the 5 year WMA table were $9,000, $10,400, and $12,000, respectively.


In [ ]:
print(response.source_nodes[0].get_content())

In [ ]:
query_str = "Tell me about the discounting factors in year 5 for the 3 year WMA"
response = query_engine.query(query_str)
base_response = base_query_engine.query(query_str)

In [ ]:
print("******* LlamaParse RAG *******")
print(str(response))
print("******* Naive RAG *******")
print(str(base_response))

******* LlamaParse RAG *******
The discounting factor in year 5 for the 3-year Weighted Moving Average (WMA) is 0.7561.
******* Naive RAG *******
The discounting factor in year 5 for the 3-year Weighted Moving Average is 0.6575162324319883.


In [ ]:
query_str = "Tell me about the projected net cash flow in years 7-9 for the 5 year WMA"
response = query_engine.query(query_str)
base_response = base_query_engine.query(query_str)

In [ ]:
print("******* LlamaParse RAG *******")
print(str(response))
print("******* Naive RAG *******")
print(str(base_response))

******* LlamaParse RAG *******
The projected net cash flow for years 7 to 9 in the 5-year Weighted Moving Average scenario is as follows: Year 7 is $29,817.78, Year 8 is $30,177.48, and Year 9 is $30,469.23.
******* Naive RAG *******
The projected net cash flow for years 7 to 9 in the 5-year weighted moving average scenario is as follows: Year 7 is $29,093.33, Year 8 is $29,817.78, and Year 9 is $30,177.48.
